<a href="https://colab.research.google.com/github/Karthick47v2/mock-buddy/blob/main/Data_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install kaggle
!pip3 install kaggle

# copy kaggle.json to required dir and give permission
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# download dataset
!kaggle datasets download selfishgene/youtube-faces-with-facial-keypoints

In [ ]:
# unzip
!unzip youtube-faces-with-facial-keypoints.zip

In [9]:
# import libs
# data visualization
import pandas as pd
# easy file access
import glob
# np array
import numpy as np
# plot
import matplotlib.pyplot as plt
# image processing
import cv2

In [10]:
# read csv
csv_file = pd.read_csv('/content/youtube_faces_with_keypoints_full.csv')

In [ ]:
csv_file.head(10)

In [ ]:
print("Total videos : ", csv_file.shape[0])
print("Unique videos : ", len(csv_file['personName'].unique()))

In [11]:
# make sure there are no unwanted npz files in the directory coz we are going to search the whole dir for npz files
# using iglob to prevent unnecessary load to RAM
npz_path_list = glob.glob('/content/**/*.npz', recursive=True)

In [12]:
# get all video id
# npzPathList contains path -> xxxx/xxxx/xxx/yyy.npz....we need yyy ...
# inorder to get that
# get yyy.npz (split by '/' and get last element)
# get yyy (split by '.' and get 1st element)
vidIDs = [x.split('/')[-1].split('.')[0] for x in npz_path_list]

In [13]:
# now map video id with full path (csv contains vidID so we need it to map with path)
vid_paths = {}

for vidID, vid_path in zip(vidIDs, npz_path_list):
  vid_paths[vidID] = vid_path

In [14]:
# just making sure no data is missing
csv_file = csv_file.loc[csv_file.loc[:, 'videoID'].isin(vid_paths.keys()), :].reset_index(drop=True)

In [ ]:
# get hands-on the data AKA play with data
# visualize 4 frames of 4 different videos with landmarks
np.random.seed(42)
num_vids = 4
num_frames = 4
frames_list = np.array([0.1, 0.3, 0.6, 0.9])

# get rand vid ids
sample_vids = csv_file.loc[np.random.choice(csv_file.index, size=num_vids, replace=False), 'videoID']

In [ ]:
# visualize data using matplotlib
fig, arr = plt.subplots(nrows=num_vids, ncols=num_frames, figsize=(14,18))

for i, videoID in enumerate(sample_vids):
  # load nparray
  np_file = np.load(vid_paths[videoID])
  # it contains colorimages, boundingbox, landmarks2D
  # color image shape is 4 (x, y, RGB, no_of_frames)
  col_img = np_file['colorImages']
  landmarks = np_file['landmarks2D']

  # generate color image from np array (uin8 -> 0 - 255)
  frames = (frames_list * (col_img.shape[3] - 1)).astype(np.uint8)

  # generate image, landmarks
  for j, frame in enumerate(frames):
    arr[i][j].imshow(col_img[:, :, :, frame])
    arr[i][j].scatter(x=landmarks[:,0,frame], y=landmarks[:, 1, frame], s=3, c='r')

In [ ]:
# save resized images and resized landmarks separately for future usage
# total 2194 -- but i am taking 2000 video files and getting each 45th frame from each video (0, 45, 90..)
# so total training datatset length ~ 6350 (192x192x3 np format)
frames = []
landmark_xy = []
img_size = 192     # resize all frames to 192 x 192
frame_cut = 45

for i, vidID in enumerate(csv_file['videoID']):
  # as it contains each frames of video, we can omit many frames 
  # here i am only using 1 frame per video

  if i == 2000:
    break

  np_file = np.load(vid_paths[vidID])
  col_img = np_file['colorImages']
  landmarks2D = np_file['landmarks2D']
  
  n = int(((col_img.shape[3] - 1) / frame_cut) + 1)
  for j in range(n):
    frame_h, frame_w = col_img[:, :, :, j * frame_cut].shape[:2]
    scale_h, scale_w = img_size / frame_h, img_size / frame_w

    landmarks = landmarks2D[:, :, j * frame_cut]
    landmarks[:, 0] = landmarks[:, 0] * scale_w
    landmarks[:, 1] = landmarks[:, 1] * scale_h 
    landmark_xy.append(landmarks)

    frames.append(cv2.resize(col_img[:, :, :, j * frame_cut], (img_size, img_size)) / 255)

frames = np.array(frames)
landmark_xy = np.array(landmark_xy)

In [11]:
# arrage to model train format
y_data = landmark_xy.reshape(landmark_xy.shape[0], -1)
y_train = np.reshape(y_data, (-1, 1, 1, 136)) / img_size

In [ ]:
#check 
fig, arr = plt.subplots(nrows=1, ncols=4, figsize=(15,15))

for i in range(4):
  arr[i].imshow(frames[i])
  x = np.reshape(y_train[i, :, :, np.arange(0, 136, 2)], (68)) * img_size
  y = np.reshape(y_train[i, :, :, np.arange(1, 136, 2)], (68)) * img_size
  arr[i].scatter(x, y, s=3, c='r')

In [13]:
# save np array for future use (training model)
np.save('frames45.npy', frames)
np.save('y_train45.npy', y_train)

In [ ]:
# move the saved files to google drive
from google.colab import drive
drive.mount('/content/gdrive')

In [6]:
!mv y_train45.npy frames45.npy /content/gdrive/MyDrive/data-v/

In [15]:
# save resized images and resized landmarks separately for test dataset
# here taking only 1 frame from each vid file. (upto 100 vids)
test_frames = []
test_bbox = []
img_size = 192     # resize all frames to 192 x 192

for i, vidID in enumerate(csv_file['videoID']):
  # as it contains each frames of video, we can omit many frames 
  # here i am only using 1 frame per video

  if i == 100:
    break

  np_file = np.load(vid_paths[vidID])
  col_img = np_file['colorImages']
  bbox = np_file['boundingBox']
  frame_h, frame_w = col_img[:, :, :, 0].shape[:2]
  scale_h, scale_w = img_size / frame_h, img_size / frame_w

  box = bbox[:,:,3]
  box[:,0] = box[:,0] * scale_w
  box[:,1] = box[:,1] * scale_h
  test_bbox.append(box)

  test_frames.append(cv2.resize(col_img[:, :, :, 3], (img_size, img_size)) / 255)

test_frames = np.array(test_frames)
test_bbox = np.array(test_bbox)

In [ ]:
fig,arr = plt.subplots(nrows=1,ncols=4,figsize=(15,15))
for i in range(4):
    arr[i].imshow(test_frames[i])
    arr[i].scatter(test_bbox[i,:,0],test_bbox[i,:,1],s=5)



In [17]:
# save np array for future use (testing model)
np.save('test_frames.npy', test_frames)
np.save('test_bbox.npy', test_bbox)

In [18]:
!mv test_frames.npy test_bbox.npy /content/gdrive/MyDrive/data-v/